In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
import torch
from rich import print as rprint
from torch import nn
from torch.utils.data import DataLoader

from analysis.common import load_model
from analysis.residual_alignment_methods import alignment, plotsvals, sab, trajectories

# from koopmann import aesthetics
from koopmann.data import (
    DatasetConfig,
    get_dataset_class,
)
from koopmann.models import MLP, ResMLP
from koopmann.utils import (
    compute_model_accuracy,
    get_device,
)
from koopmann.visualization import plot_decision_boundary

%load_ext autoreload
%autoreload 2

In [2]:
file_dir = "/scratch/nsa325/koopmann_model_saves"
model_name = "resmlp"
device = get_device()

In [3]:
model, model_metadata = load_model(file_dir, model_name)
model.hook_model()
print(model_metadata)

{'batchnorm': True, 'bias': False, 'created_at': '2025-03-19T00:28:50.619948', 'dataset': 'MNISTDataset', 'hidden_config': [784, 784, 784, 784], 'in_features': 784, 'model_class': 'ResMLP', 'nonlinearity': 'relu', 'out_features': 10, 'stochastic_depth_mode': 'batch', 'stochastic_depth_prob': 0.0}


In [4]:
# Dataset config
dataset_config = DatasetConfig(
    dataset_name=model_metadata["dataset"],
    num_samples=3_000,
    split="test",
    seed=42,
)
DatasetClass = get_dataset_class(name=dataset_config.dataset_name)
dataset = DatasetClass(config=dataset_config)
dataloader = DataLoader(dataset, batch_size=1000)

In [5]:
rprint(f"Testing Accuracy: {compute_model_accuracy(model, dataloader, device)}")

Testing Accuracy: 0.9901999831199646

In [6]:
if dataset.in_features == 2:
    plot_decision_boundary(
        model,
        model.state_dict(),
        dataset.features,
        dataset.labels.squeeze(),
        labels=[0, 1, 2],
    )